### importing jars

In [1]:
%classpath add jar ../../resources/libs/component-catalog.jar
%classpath add jar ../../resources/libs/weka-3.8.2.jar
%classpath add jar ../../resources/libs/xchart-3.5.2.jar
%classpath add jar ../../resources/libs/mqttv3-1.1.2.jar

In [7]:
package pt.c08componentes.s20catalog.s90ml;

import java.util.Random;

import pt.c08componentes.s20catalog.s00shared.IColumnProducer;
import pt.c08componentes.s20catalog.s00shared.IInstancesProducer;
import weka.classifiers.Evaluation;
import weka.classifiers.trees.J48;
import weka.core.Instance;
import weka.core.Instances;

public class DiagnosticsComponent implements IDiagnostics {
  private IInstancesProducer iProducer;
  IColumnProducer cProducer;
  IEHR ehrOutput;
  
  public void connect(IInstancesProducer producer) {
    iProducer = producer;
  }
  
  public void connect(IEHR ehr) {
    this.ehrOutput = ehr;
  }
	
  public void diagnose(String name, int age, double[] symptoms) {
	sendEHR(new EHR(name, age, symptoms, ""));
  }
  
  public void sendEHR(EHR ehrInput) {
    try {
      if (iProducer != null) {
        Instances dataTrain = iProducer.requestInstances();
        // setting class attribute
	    dataTrain.setClassIndex(dataTrain.numAttributes() - 1);
	    
	    System.out.println(dataTrain.instance(0));
	      
	    String[] options = {"-U"};  // unpruned tree
	    J48 tree = new J48();       // new instance of tree
	    tree.setOptions(options);   // set the options
	    tree.buildClassifier(dataTrain); // build classifier
	      
	    Evaluation eval = new Evaluation(dataTrain);
	    eval.crossValidateModel(tree, dataTrain, 10, new Random(1)); 
	      
	    System.out.println(eval.toSummaryString("\nResults\n======\n", false));

	    Instance input = dataTrain.instance(0).copy(ehrInput.getSymptoms());
        double clsLabel = tree.classifyInstance(input);
        input.setClassValue(clsLabel);
        
//         ehrInput.setDiagnostics(input.toString());
//         if (ehrOutput != null)
//         	ehrOutput.sendEHR(ehrInput);
	      
        
        System.out.println("===== AA   Classification:");
	    System.out.println(input);
	    
	    
      }
    } catch (Exception e) {
      e.printStackTrace();
    }
  }
}


pt.c08componentes.s20catalog.s90ml.DiagnosticsComponent

In [8]:
import pt.c08componentes.s20catalog.s10ds.DataSetComponent;
import pt.c08componentes.s20catalog.s10ds.IDataSet;
import pt.c08componentes.s20catalog.s90ml.AnamnesisRegisterComponent;
import pt.c08componentes.s20catalog.s90ml.DiagnosticsComponent;
import pt.c08componentes.s20catalog.s90ml.EHRConsoleComponent;
import pt.c08componentes.s20catalog.s90ml.IAnamnesisRegister;
import pt.c08componentes.s20catalog.s90ml.IDiagnostics;
import pt.c08componentes.s20catalog.s90ml.IEHRConsole;

try {
    IAnamnesisRegister anamnesis = new AnamnesisRegisterComponent();

    IDiagnostics diagnostics = new DiagnosticsComponent();
    anamnesis.connect(diagnostics);

    IEHRConsole console = new EHRConsoleComponent();
    diagnostics.connect(console);
    
    double[] symptoms = {1, 1, 1, 1, 1, 1, 1, 1};
    
    diagnostics.diagnose("Asdrubal", 30, symptoms);

    IDataSet dataset = new DataSetComponent();
    dataset.setDataSource("../../resources/datasets/zombie-health-spreadsheet-ml-training.arff");
    diagnostics.connect(dataset);

    anamnesis.start();
    
} catch (Exception e) {
    e.printStackTrace();
}

t,t,f,f,f,f,f,bacterial_infection


Aug 11, 2018 6:42:54 PM com.github.fommil.netlib.ARPACK <clinit>
Aug 11, 2018 6:42:54 PM com.github.fommil.netlib.ARPACK <clinit>
Aug 11, 2018 6:42:54 PM com.github.fommil.netlib.ARPACK <clinit>
Aug 11, 2018 6:42:54 PM com.github.fommil.netlib.ARPACK <clinit>



Results

Correctly Classified Instances          14               73.6842 %
Incorrectly Classified Instances         5               26.3158 %
Kappa statistic                          0.6455
Mean absolute error                      0.1745
Root mean squared error                  0.3477
Relative absolute error                 56.0569 %
Root relative squared error             87.2821 %
Total Number of Instances               19     

===== AA   Classification:
t,t,f,f,f,f,f,bacterial_infection


null

In [4]:
import pt.c08componentes.s20catalog.s10ds.DataSetComponent;
import pt.c08componentes.s20catalog.s10ds.IDataSet;
import pt.c08componentes.s20catalog.s20console.ConsoleComponent;
import pt.c08componentes.s20catalog.s20console.IConsole;

try {
    
    IDataSet dataset = new DataSetComponent();
    dataset.setDataSource("../../resources/datasets/zombie-health-spreadsheet-ml-training.csv");

    IConsole console = new ConsoleComponent();
    console.connect(dataset);

    console.update();

} catch (Exception e) {

    e.printStackTrace();

}


@relation zombie-health-spreadsheet-ml-training

@attribute paralysis {t,f}
@attribute yellow_tong {t,f}
@attribute member_loss {f,t}
@attribute chest_pain {f,t}
@attribute trembling_finger {f,t}
@attribute severe_anger {f,t}
@attribute history_bacteria {f,t}
@attribute diagnostic {bacterial_infection,bite_deficit,viral_infection,fights,nothing}

@data
t,t,f,f,f,f,f,bacterial_infection
f,t,f,f,f,f,f,bacterial_infection
f,t,f,f,t,f,t,bite_deficit
f,t,t,f,t,f,f,bite_deficit
f,f,t,t,f,f,f,viral_infection
f,f,t,f,f,t,f,fights
f,f,f,f,f,t,f,nothing
f,f,f,f,t,f,f,bite_deficit
f,t,f,t,f,f,f,bacterial_infection
f,f,f,t,f,f,f,viral_infection
f,t,t,f,f,f,t,bite_deficit
t,t,f,f,f,f,f,bacterial_infection
f,f,f,t,f,f,t,viral_infection
f,f,t,f,f,f,f,fights
f,t,f,f,t,f,t,bite_deficit
f,t,t,f,t,f,f,bite_deficit
f,f,f,t,f,f,f,fights
f,t,f,f,f,f,f,bacterial_infection
f,f,f,f,t,f,f,bite_deficit


null

In [5]:
import org.eclipse.paho.client.mqttv3.IMqttDeliveryToken;
import org.eclipse.paho.client.mqttv3.MqttCallback;
import org.eclipse.paho.client.mqttv3.MqttClient;
import org.eclipse.paho.client.mqttv3.MqttException;
import org.eclipse.paho.client.mqttv3.MqttMessage;

public class PahoDemo implements MqttCallback {

MqttClient client;

public PahoDemo() {
}

public static void main(String[] args) {
    new PahoDemo().doDemo();
}

public void doDemo() {
    try {
        System.out.println("entrei");
        client = new MqttClient("tcp://localhost:1883", "Sending");
        client.connect();
        client.setCallback(this);
        client.subscribe("foo");
        MqttMessage message = new MqttMessage();
        message.setPayload("A single message from my computer fff"
                .getBytes());
        client.publish("foo", message);
    } catch (MqttException e) {
        e.printStackTrace();
    }
}

@Override
public void connectionLost(Throwable cause) {
    // TODO Auto-generated method stub

}

@Override
public void messageArrived(String topic, MqttMessage message)
        throws Exception {
        MqttMessage message2 = new MqttMessage();
        message2.setPayload("recebi a parada"
                .getBytes());
        client.publish("foo2", message2);
}

@Override
public void deliveryComplete(IMqttDeliveryToken token) {
    // TODO Auto-generated method stub

}

}

com.twosigma.beaker.javash.bkr38696e6b.PahoDemo

In [6]:
PahoDemo.main(new String[5]);

entrei


null